In [118]:
import pynq
import time
import numpy as np
import socket
import pickle

In [119]:
HOST = "192.168.0.160"  # The server's hostname or IP address
PORT = 65434  # The port used by the server

In [120]:
bitstream = "real_matmul_optimized.bit"
overlay = pynq.Overlay(bitstream)
dut = overlay.real_matmul_0

In [121]:
A = pynq.allocate((100, 150), dtype='int16')
B = pynq.allocate((150, 200), dtype='int16')
C = pynq.allocate((100, 200), dtype='int16')

In [122]:
arr=[]
arr1=[]
arr2=[]
arr3=[]

time.sleep(11)
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    
    s.connect((HOST, PORT))
    for i in range(1000):
        result_bytes = s.recv(2*15)
        result = np.frombuffer(result_bytes, dtype=np.int16)
        arr= result.copy()
        arr1.append(arr)
        
    for i in range(1000):
        result_bytes = s.recv(2*30)
        result = np.frombuffer(result_bytes, dtype=np.int16)
        arr= result.copy()
        arr2.append(arr)

    
#    result_bytes = s.recv((2*30000))
#    result = np.frombuffer(result_bytes, dtype=np.int16)
#    arr2= result.copy()
    
#    result_bytes = s.recv((2*15001))
#    result = np.frombuffer(result_bytes, dtype=np.int16)
#    arr3.append(result)
    
#    time.sleep(5)
    
arr1 = [x for y in arr1 for x in y]
arr2 = [x for y in arr2 for x in y]

arr1 = np.array(list(arr1), dtype='int16')
arr2 = np.array(list(arr2), dtype='int16')

arr1= arr1.reshape(100,150)
arr2= arr2.reshape(150,200)

print("1st Array received: ", (arr1))
print("2nd Array received: ", (arr2))


1st Array received:  [[990 682 390 ... 778 374 954]
 [731  52 727 ... 970  39 665]
 [894 483 353 ... 369  73 232]
 ...
 [232 673 900 ... 259  98 576]
 [500 653 828 ... 280 113 954]
 [382 199 501 ... 133 831 527]]
2nd Array received:  [[ 76 304 743 ... 777  23 724]
 [754 427 329 ... 274 543 320]
 [738 388 130 ... 688 560 150]
 ...
 [853 493 170 ...  50 902 562]
 [688 561 953 ... 478 317 114]
 [720 910 678 ... 355 640 224]]


# Create a socket object
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Define the server address and port
server_address = (HOST, PORT)

# Bind the socket to the server address
sock.bind(server_address)

# Listen for incoming connections
sock.listen(1)

# Accept a client connection
print("Waiting for a client connection...")
client_socket, client_address = sock.accept()
print("Client connected:", client_address)

# Receive the serialized array from the client
serialized_array = client_socket.recv(4096)

# Deserialize the array using pickle
array = pickle.loads(serialized_array)

# Print the received array size
print("Received array size:", len(array))

# Close the client socket
client_socket.close()

# Close the server socket
sock.close()


In [123]:
for i in range(100):
    for j in range(150):
        A[i][j]= arr1[i][j]

for i in range(150):
    for j in range(200):
        B[i][j]= arr2[i][j]

In [124]:
A.sync_to_device()
B.sync_to_device()
dut.register_map
dut.register_map.MatA_DRAM_1 = A.device_address & 0xFFFFFFFF
dut.register_map.MatA_DRAM_2 = A.device_address >> 32

dut.register_map.MatB_DRAM_1 = B.device_address & 0xFFFFFFFF
dut.register_map.MatB_DRAM_2 = B.device_address >> 32

dut.register_map.MatC_DRAM_1 = C.device_address & 0xFFFFFFFF
dut.register_map.MatC_DRAM_2 = C.device_address >> 32
start_time = time.time()

dut.register_map.CTRL.AP_START = 1
dut.register_map.CTRL[4] = 1
while not dut.register_map.CTRL.AP_DONE: pass

end_time = time.time()
duration = end_time - start_time
print(f'Kernel completed in {duration * 1000:.2f}ms')
C.sync_from_device()

Kernel completed in 3.56ms


In [125]:
C_expected= np.matmul(arr1,arr2)

In [126]:
passed=1
for i in range(100):
    for j in range(200):
        if(C_expected[i][j]!= C[i][j]):
            print("Mismatch at C_expected["+str(i)+"]["+str(j)+"]: Expected: "+str(C_expected[i][j])+" \t Actual: "+str(C[i][j])+"\n")
            passed=0

if(passed):
    print("-----------------------------------\n");
    print("|         TEST PASSED!            |\n");
    print("-----------------------------------\n");

else:
    print("-----------------------------------\n");
    print("|         TEST FAILED :(          |\n");
    print("-----------------------------------\n");

-----------------------------------

|         TEST PASSED!            |

-----------------------------------



In [127]:
overlay.free()

In [128]:
start_time = time.time()

C_out = np.matmul(A, B)

end_time = time.time()
duration = end_time - start_time
print(f'PS computation completed in {duration * 1000:.2f}ms')

PS computation completed in 277.66ms


In [129]:
#Sending the data back
c= np.empty(shape=(100,200))
for i in range(100):
    for j in range(200):
        c[i][j]= C[i][j]

c_ser= c.flatten()
c_1_bytes = np.array(c_ser, dtype=np.int16).tobytes()
#c_2_bytes = np.array(c_ser[10000:20000], dtype=np.int16).tobytes()

print("Array Sent: ", c_ser)

PORT = 6002
time.sleep(15)
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    
    s.connect((HOST, PORT))

    data= c_1_bytes
    s.sendall(data)
        
#    data= c_2_bytes
#    s.sendall(data)


Array Sent:  [-12746.   6688.  29355. ... -11491.  14565.   3814.]
